# structurer du texte et automatiser la création d'un fichier HTML à partir d'un texte brut

---

## Introduction

Après un rappel des bases dans `text2web_intro`, on rentre dans le vif du sujet ! Dans cette première partie, on va s'occuper de la partie "texte": traiter le texte source afin de le structurer et en faire une sortie HTML.

**Le plan d'attaque** : 
- ouvrir le fichier à partir duquel on travaille
- le parser, c'est-à-dire lui donner une structure à partir de laquelle on pourra travailler
- construire du HTML en masse à partir du texte structuré
- ajouter ce HTML à un squelette HTML prééxistant et enregistrer le tout dans une sortie.

---

## Ouvrir le fichier

On commence par ouvrir le fichier et stocker son contenu dans une variable pour pouvoir travailler dessus. Python peut lire, écrire et mettre à jour des fichiers texte. Pendant ce tutoriel, nous allons nous familiariser avec deux options: lecture et écriture de fichier.

Il y a deux manières pour ouvrir un fichier; dans les deux cas on s'appuie sur `open()`:
- avec `fh = open("chemin/vers/fichier", mode="...")`. Dans ce cas, le fichier reste ouvert jusqu'à ce qu'on le ferme avec `fh.close()`. Il ne jamais garder des fichiers ouverts si on ne se sert pas de leur contenu! Ça utilise beaucoup de mémoire et augmente le risque d'erreurs.
- avec `with open("chemin/vers/fichier", mode="...") as fh`. Dans ce cas, on revient à la ligne, on indente et le fichier se ferme automatiquement dès qu'on désindente (voir l'introduction sur l'indentation).

Dans les deux cas, `open()` fonctionne de la même manière: on ouvre un fichier et on l'attribut à une variable (`fh`)
```python
with open("chemin/vers/fichier", mode="...", encoding="...") as fh:
    # code à faire avec le fichier ouvert
```
- Les **attributs** de la fonction sont:
    - `chemin/vers/fichier`: le chemin relatif depuis l'endroit où on exécute le script vers le fichier que l'on veut ouvrir
    - `mode=""`: ce qu'on veut faire avec le fichier:
        - `r`: lire le fichier
        - `w`: écrire dans un fichier; créer le fichier s'il n'existe pas. supprime tout le contenu déjà présent dans le fichier
        - `a`: ajouter du contenu à la fin d'un fichier prééexistant
        - ...
    - `encoding=`: l'encodage du fichier. içi, on utilisera `utf-8`, l'encodage par défaut.

Ouvrons donc notre fichier et lisons son contenu.

In [ ]:
with open("./input/source.txt", mode="r", encoding="utf-8") as fh:
    source = fh.read()  # on lit le contenu avec file.read()
    
# si on veut tester si un fichier est ouvert en dehors de l'indentation,
# on exécute la ligne ci-dessous et on va avoir une jolie erreur.
# print(fh.read())

In [ ]:
# maintenant, on imprime le début du texte la structure et le contenu
# on imprime les 2 premières entrées, le fichier complet est bien plus lourd
print(source[0:365])

## Analyser la structure

Regardez bien le texte ci-dessus. Qu'est-ce que vous appercevez ?

- D'abord, on peut voir de quoi parle notre texte: c'est un catalogue de ventes de manuscrits.
- Ensuite, ce qui est plus intéressant: la structure du texte. On a plusieurs séparateurs qui se répètent et qui séparent les différentes parties du texte (comme quoi, le monde est bien fait. à croire que j'ai fait exprès de préparer le document comme ça). Quels sont ces séparateurs ?
    - `\n\n\n`: un triple saut de lignes, qui permet de séparer les différentes pièces en vente
    - `\n`: un retour à la ligne simple, qui permet de séparer les différentes partie d'une même entrée de catalogue. On va dire qu'il y a 3 parties: titre de la vente, descriptif et prix.
    - `--`: un double tiret qui sépare le numéro de l'item vendu du nom de la personne
    - `~`: un tilde qui sépare le nom de la personne de sa description.
    
Quand on fait du traitement de masse, la première étape est toujours de comprendre à quoi on a à faire. En tant qu'humain.e.s, on a tendance à appréhender un texte à partir de son contenu. Mais un ordinateur ne peut pas comprendre le contenu (même quand on fait de l'apprentissage machine). Il faut donc aussi trouver un moyen de dériver une structure abstraite pour le texte que l'on va traiter. Grâce aux séparateurs ci-dessus, on comprend qu'on a cette structure:
```
    numéro de vente -- nom de la personne ~ descriptif de la personne \n
    descriptif de l'item vendu \n
    prix \n\n\n
```
On a une structure. On peut donc travailler avec ça. Dans un premier temps, on va partir de cette structure pour scindre le texte en unités/sous-unités à partir desquelles on pourra structurer notre HTML. Pour ce faire, on va représenter le texte sous forme de listes imbriquées. Au final, on veut que notre texte soit exprimé de cette manière:
```python
    [
        [titre1, description1, prix1],
        [titre2, description2, prix2],
        [titreN, descriptionN, prixN]
    ]
```
N.b.: le meilleur format pour représenter des données structurées de manière simple reste le dictionnaire. Ici, on travaille sur des données très simples et relativement propres, donc on travaillera avec des listes imbriquées, qui sont plus faciles à manipuler qu'un dictionnaire.

---

## Structurer notre texte

On va donc écrire un script qui structure notre texte brut. Pour commencer, on crée une variable pour stocker la liste:

In [ ]:
source_tolist = []  # le texte brut sous forme de liste

Ensuite, on commence à **scinder le texte en unités/sous-unités**. Pour ce faire, il faut partir de la plus grande unité signifiante (l'item) pour aller aux plus petites (titre, description, prix). 

Pour transformer une chaîne de caractère en liste, on utilise **`chaîne.split()`**. 
- Par défaut, `.split()` sépare les chaînes mot par mot:

In [ ]:
chaine = "Kasimir Malevitch|Natalia Gontcharova"
print(chaine.split())

- Mais on peut spécifier un caractère particulier en attribut de `.split()`:

In [ ]:
print(chaine.split("|"))

On commence donc par séparer notre texte source par un triple saut de ligne, afin de pouvoir accéder aux entrées de catalogue individuellement:

In [ ]:
items = source.split(r"\n\n\n")  # le r"" permet d'éviter que python échappe les caractères
print(items[0:2])

On a donc maintenant items, une liste de chaque entrée de catalogue. Maintenant, on va devoir créer une structure interne aux catalogues. Ici, la méthode est analogue: 

- on se débrouille pour accéder à chaque entrée de catalogue individuellement
- on prend notre séparateur suivant (`\n`, le saut de ligne unique), et on transforme chaque entrée de catalogue en liste avec ce saut de ligne.
- pour finir, il faudra ajouter à notre variable de sortie `source_tolist` les items séparés bien comme il faut pour enfin avoir notre texte sous la forme de listes imbriquées. Pour rappel, le résultat attendu est:
```
    [
        [titre1, description1, prix1],
        [titre2, description2, prix2],
        [titreN, descriptionN, prixN]
    ]
```

Avec notre introduction et tout ce qu'on a déjà vu, vu devez être capable de faire ça tout.e seul.e !

In [ ]:
# EXERCICE 2 : SCINDER CHAQUE ITEM DE LISTE EN SOUS-PARTIES AVEC LE SÉPARATEUR \n

# à vous de jouer...

## Automatiser la création de HTML

On a maintenant un minumum de structure à partir de laquelle on pourra travailler pour construire un joli fichier HTML. Pour cette étape, notre **plan d'attaque** sera:
- accéder à chaque item individuellement
- pour chaque item, construire une structure HTML qui reprenne la structure du document originel
- ajouter l'HTML de chaque item à une variable qui contienne les représentations HTML de chacun de nos items
- ajouter ce gros HTML à un squelette déjà prêt pour avoir un HTML complet et valide
- écrire le gros fichier HTML en sortie.

La structure HTML attendue pour chaque entrée de catalogue est la suivante:
```html
<div>
    <dt>Item n°XXX </dt>
    <dd>Nom de la personne: <i>description de la personne</i></dd>
    <ul>
        <li>Description de l'item en vente</li>
        <li>XXX francs.</li>
    </ul>
</div>
```
Chacun de ces `<div>` viendra s'ajouter à un autre `<div>` et finira dans un gris fichier HTML. Pour information, voilà le sens des différents éléments utilisés:
- `<div>` : un simple conteneur, qui est censé contenir directement du texte, et pas d'autres éléments
- `<dd>` et `<dt>`: couple qui permet d'avoir un élément en gras et une description de l'élément en gras.
- `<ul>`: conteneur pour une liste (à puces par défaut)
- `<li>`: élément de la liste

C'est peut-être un peu beaucoup de vous laisser faire toute la génération de HTML à ce stade. Mais il faut garder en tête que:
- on a une liste structurée qui correspond à nos différents éléments (à l'exception du `titre`, que l'on devra scinder en `<dt>`, `<dd>` et `<i>`). Il faudra donc trouver un moyen de faire la différence entre les différents éléments en repérant des motifs. Ne pas hésiter à aller regarder le document source pour bien comprendre.
- du HTML, c'est seulement une chaîne de caractères

Dès lors, la seule chose qu'il y a à faire, c'est, en fonction de l'élément sur lequel on travaille, il faut ajouter un certain type de texte, c'est à dire concaténer des chaînes de caractères. Avant de continuer, pouvez-vous, en écrivant du "faux" code, **détailler le processus à suivre pour partir de la liste contenant nos entrées et arriver à une variable contenant tous les éléments en HTML** ? Le code ne doit pas nécessairement marcher, mais juste décrire le processus à suivre en language python.

In [ ]:
# EXERCICE 3 - ÉCRIRE UN PROCESSUS EN "PSEUDO CODE" QUI DÉTAILLE LE PROCESSUS POUR
# CONSTRUIRE LE HTML COMPLET

# pas de correction ici, vu qu'on détaille le processus (et donc la réponse) en dessous

Comme on l'a vu, le **processus** n'est pas extrêmement compliqué: 
- accéder individuellement à chaque sous-liste dans notre liste principale (c'est-à-dire, à chaque entrée de catalogue) 
- itérer sur cette sous-liste pour pouvoir traiter les entrées de catalogue indivudellement
- pour chaque item de la sous-liste, trouver un marqueur qui permette d'identifier le type d'information contenue (titre, description, prix).
    - si l'item est un titre, faire séparer le titre en 3 éléments: le n° d'item, le nom et sa description. Concaténer chacun de ces éléments avec les bonnes balises HTML
    - si l'item est une description ou un prix, ajouter les bonnes balises HTML.
    
Maintenant, identifions nos **marqueurs** qui permettent d'identifer les parties d'une entrée de catalogue. Pour ça, pas besoin de python, juste de connaître son document source.
- un **titre** contient `--` et `~`. Ça tombe bien, ces deux marqueurs séparent le numéro d'item, le nom et sa description.
- un **prix contient** un nombre suivi de `francs.`. On peut donc utiliser `francs.` comme séparateur
- par élimination, la **description** est ce qui n'est ni un prix, ni un titre. Il n'y a donc pas besoin de trouver un marqueur spécifique (même si on pourrait).

Au passage: identifier des marqueurs, c'est faire de la **détection de motifs** ([*pattern recognition*](https://en.wikipedia.org/wiki/Pattern_recognition)). Cela revient à reprérer dans la *structure* (c'est-à-dire dans la forme, pas dans le sens) du texte des éléments qui se répètent. Ça tombe en dehors du spectre de ce tutoriel, mais pour faire des choses un peu plus intéressantes en détection de motif, les [expressions régulières](https://fr.wikipedia.org/wiki/Expression_r%C3%A9guli%C3%A8re) sont particulièrement utiles. Il y a beaucoup de fiches et d'exercices en ligne pour s'entraîner!

On a le processus et les marqueurs à utiliser, il ne reste plus que **l'implémentation pratique** (en python qui marche, cette fois-ci) !
- on a parlé d'accéder aux items d'une sous liste. Il va donc falloir faire une boucle dans une boucle.
- ensuite, on doit détecter des motifs; en fonction de ces motifs, notre script doit faire différentes actions. On va donc aussi devoir utiliser un `if...else`.

Histoire de se mettre dans l'ambiance, voyons comment fonctionne une double itération. `type(donnée)` permet de voir le type (`str`, `list`, `dict`) de la donnée avec laquelle on travaille. C'est une fonction utile pour savoir où on en est et pour débuguer.

In [ ]:
print(source_tolist[:3])  # les 3 premières entrées
print(type(source_tolist))  # le type de la donnée sur laquelle on travaille (une liste)
print(len(source_tolist[:3]))
print("_______________________________________________________________________")

nloop = 0  # variable pour compter le nb d'itérations et éviter de boucler sur toute la liste

# première itération. qu'est-ce qui se passe ici ?
for item in source_tolist:
    if nloop < 3:  # on imprimera que les 3 entrées grâce à notre compteur d'itérations
        nloop += 1  # on incrémente nloop. syntaxe équivalente à nloop = nloop + 1
        print(item)
        print(type(item))
        print("...................................")
        
        # deuxième itération à l'intérieur de la première. qu'est-ce qui se passe ?
        for part in item:
            print(part)
            print(type(part))
            print("...................................")
        print("_______________________________________________________________________")
        

In [ ]:
html = ""  # structure d'accueil pour le html complet

# on récupère la variable source_tolist créée dans l'exercice 2
# on itère sur tous les items pour accéder à chaque item (entrée de catalogue) individuellement
for item in source_tolist:
    item_html = "<div>"  # variable pour stocker l'html de l'item
    list_html = "<ul>"  # variable pour stocker une liste description-prix de chaque item
    
    # on itère sur chaque sous-liste (c-a-d, chaque partie de l'entrée)
    for part in item:  
        # ensuite, il faut déterminer sur quel type d'entrée en travaille,
        # c'est-à-dire repérer un motif présent dans toutes les entrées.
        # dans ce cas ne pas hésiter à jeter un œil au document source.
        # les choses sont bien faites, on a prévu qu'il y ait des motifs
        # dans toutes les parties:
        # - "--" et "~" permettent d'identifier que la "part" est le titre 
        #   (et de faire la séparation entre les différentes parties du titre)
        # - tous les prix sont suivis de la mention "francs."
        # - par élimination, la 3e "part" de chaque "item" est une description
        # on a donc:
        if "--" in part:
            # c'est un titre. il va falloir le séparer en 3 parties:
            # n° d'item, nom et description de la personne
            # on commence par séparer le numéro du reste
            part = part.split("--")
            numero = part[0]
            title = part[1]
            
            # ensuite, on sépare title entre nom et description
            title = title.split("~")
            name = title[0]  # le nom vient en première partie du titre
            # pour extraire la description (trait), on fait les
            # choses de manière élégante: il n'y a pas toujours
            # de description. donc, on choisir de ne rajouter un élément <i>
            # seulement si il y a une description
            if len(title) > 1:
                trait = title[1]
            else:
                trait = ""
                
            # on complète la chaîne item_html avec le titre
            # la syntaxe f"" permet d'insérer des variables (écrites entre
            # {}) à l'intérieur d'une chaîne de caractère
            item_html += f"<dt>Item n°{numero}</dt><dd>{name}: <i>{trait}</i></dd>"
            
        # on passe ensuite aux prix. on considère qu'ils sont tous en francs
        elif "francs." in part:
            price = part
            list_html += f"<li>{price}</li>"
            
        # par élimination, l'item de liste restant est une descrption
        else:
            desc = part
            list_html += f"<li>{desc}</li>"
    
    # on a finit de traiter notre item. on ferme la liste, on l'ajoute à item_html
    list_html += "</ul>"
    item_html += list_html
    item_html += "</div>"  # on ferme notre item
    html += item_html  # on rajoute notre item au html contenant tous les items
    
print(html[0:482])  # on vérifie que tout va bien en imprimant 

## Écrire notre HTML dans un fichier

Notre HTML est tout beau tout propre ! Le seul problème, c'est qu'on a pas créé de document HTML entier et valide, mais seulement un bout d'HTML à aller intégrer à un document. Les choses sont bien pensées, il y a une squelette HTML d'accueil dans le dossier `utils/catalog_web_skeleton.html`. Il suffit donc de mettre le squelette à jour avec le contenu de notre variable `html`.

Il y a un petit problème: mettre à jour, c'est un concept simple pour un cerveau, mais pas pour un ordinateur. On ne peut pas "juste" dire à python d'aller rajouter le texte qu'il faut à l'endroit qu'il faut. Comme toujours avec python, quand on a un problème, on doit **décomposer** le problème en petites étapes jusqu'à ce qu'il n'y ait plus de problème. Pour mettre à jour un fichier, il faut donc:
- ouvrir le fichier à mettre à jour en lecture et stocker son contenu dans une variable
- ajouter le contenu qu'on veut ajouter à l'endroit où on veut l'ajouter, dans cette variable
- écrire le contenu de la variable mise à jour dans un fichier.

À vous ! Pour rappel:
- le fichier d'entrée est `utils/catalog_web_skeleton.html`
- dans ce fichier, il y a la chaîne de caractère `{PLACEHOLDER_BASE}`. C'est cette chaîne qui doit être remplacée par le contenu de notre variable `html`. Pour remplacer, on utilise la méthode `.replace()` :
```python
        chaine.replace(chose_à_replacer, remplacement)
```
- le document doit être enregistré dans `output/catalog_web.html`. Pour écrire un fichier, il faut 
    - l'ouvrir avec `mode="w"`
    - assigner le fichier ouvert à une variable (disons `fh`)
    - écrire dans ce fichier avec `fh.write(contenu_ou_variable_à_ecrire)`
    - **attention**: quand on ouvre un fichier avec `mode="w"`, tout son contenu est écrasé

In [ ]:
# EXERCICE 4 : ÉCRIRE LA SORTIE HTML

# avant de passer à la suite, pour être sûr.e que tout va bien, regardez la correction 
# et exécutez la si besoin pour qu'on commence le tutoriel suivant sur de bonnes bases

## En conclusion

C'est tout pour maintenant ! On a appris à ouvrir des fichiers, lire leur contenu, écrire dedans, traiter des chaînes de caractères simples, retyper (c'est-à-dire convertir un type de donnée en une autre: `str`>`list` dans notre cas), créer du HTML automatiquement... 

Vous pouvez maintenant aller voir le résultat en ouvrant dans un navigateur le fichier `output/catalog_web.html`. Il y aura un `{PLACEHOLDER_...}` en haut du fichier, mais c'est pour le mettre à jour dans la partie suivante. J'ai intégré un peu de CSS (langage utilisé pour définir le style d'une feuille HTML) pour booster un peu le look de notre HTML (le code couleur n'est pas magnifique, mais bon). De manière générale, si on fait du web, un peu de CSS (un code couleur et un bon choix de polices) peut ammener très loin ! Si vous êtes curieux.ses, on peut trouver le CSS dans `utils/static/style.css`.

Dans la partie suivante: on va apprendre à :
- nettoyer du texte brut et le préparer pour une étude statistique
- compter les occurrences des différents mots (une introduction à l'analyse statistique de texte)
- écrire, lire et manipuler du JSON
- créer des grapĥiques interactifs avec Plotly

**Exercice bonus (si vous avez le courage)**: dans ce tutoriel, on a traité notre fichier à l'aide de listes imbriquées. Dans ce cas, c'était pratique, parce que l'on travaille sur un document d'entrée très structuré, avec toujours les mêmes éléments dans chaque entrée de catalogue... Mais sur un document plus compliqué, il serait plus utile d'utiliser un dictionnaire. Essayez donc de structurer notre variable d'entrée `source` sous la forme d'un dictionnaire qui ait la structure suivante:
```python
    source_todict = {
        "item1": {
            "titre": {
                "numero": "numero de l'item",
                "nom": "nom de la personne",
                "trait": "description de la personne"
            },
            "description": "description de l'item en vente",
            "prix": "prix de l'item vendu"
        },
        "item2": ...
    }
```

Il peut y avoir quelques petites difficultés, donc voici des **points d'attention**:
- le processus de création du dictionnaire est à peu près le même que celui pour la création des listes imbriquées. La seule différence est que plutôt que d'ajouter nos bouts d'information structurés à une liste, on les assigne à des clés d'un dictionnaire.
- pour créer une entrée dans un dictionnaire, on fait: 
```python
    dictionnaire[clé] = valeur
```
- on va devoir scinder des chaînes de caractère en faisant `.split` sur des caractères spéciaux: `\n`. il faut intégrer ces caractères dans la syntaxe `r""`, qui dit à python de traiter ces caractères comme des caractères normaux
- certains items peuvent être consitués d'un seul élément vide, ce qui cause une erreur. pour éviter cela, il faut, quand on travaille au niveau de l'item, faire:
```python
    if item == " ":  # item: chaîne de caractère correspondant à une entrée de catalogue
        continue  # on saute l'élément courant si il est vide
```

In [ ]:
# EXERCICE 5 (BONUS) - exprimer `source` en dictionnaire

# en fait le processus est assez semblable à celui de la
# création d'une liste imbriquée. ce qui change, c'est qu'on assigne
# chaque partie à une clé de dictionnaire

source_todict = {}